In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/Đồ án"
!ls

In [ ]:
import glob
import os

data_path = './txt'
text_paths = sorted(glob.glob(os.path.join(data_path,'*.txt')))
docs = []

# chia batch ra, cho đỡ tốn bộ nhớ
for files in text_paths:
    with open(files, mode='r') as f:
        txt = f.read()
        docs.append(txt)

In [ ]:
import spacy
import pandas as pd
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

ne_type = nlp.get_pipe("ner").labels
des = [spacy.explain(ne) for ne in ne_type]
des = pd.DataFrame({'type': ne_type, 'des': des})
des

In [ ]:
import spacy
from spacy import displacy
from spacy import tokenizer
nlp = spacy.load('en_core_web_sm')

ne = []
labels = []

for doc in docs:        
    doc = nlp(doc)
    if doc.ents:
        for ent in doc.ents:
            ne.append(ent.text)
            labels.append(ent.label_)

df_ne = pd.DataFrame({'name': ne, 'label': labels})

In [ ]:
ne_type = nlp.get_pipe("ner").labels
# only need these entities type
ent_labels = ['EVENT','FAC', 'GPE', 'LOC', 'NORP', 'ORG', 'PERSON', 'NORP', 'PRODUCT', 'WORK_OF_ART']

In [ ]:
sub = [ne for ne in ne_type if ne not in ent_labels]
sub 

In [ ]:
df_ne = df_ne.drop_duplicates(subset = ['name'])
df_ne_sub = df_ne.loc[df_ne['label'].isin(ent_labels)]
df_ne_sub = df_ne_sub.sort_values('name')

In [ ]:
df_ne_sub['result'] = str()

In [ ]:
df_ne_sub['label'].value_counts()

In [ ]:
!pip install owlready2 SPARQLWrapper --quiet

In [ ]:
!wget -q https://raw.githubusercontent.com/grokkenTee/sparql-query/main/Data_NLP/LonelyPlanet.rar
!unrar x LonelyPlanet.rar LonelyPlanet

In [ ]:
from owlready2 import *

onto = get_ontology("./LonelyPlanet/Ontology/travel.owl").load()

In [ ]:
onto.base_iri

In [ ]:
from SPARQLWrapper import SPARQLWrapper2

def query_type(keySearch, api='http://dbpedia.org/sparql'):
    # keySearch = 'Hanoi'
    sparql = SPARQLWrapper2(api)
    sparql.setQuery(
        """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX owl: <http://www.w3.org/2002/07/owl#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        
        SELECT * WHERE
                            { ?thing rdfs:label "%s"@en . 
                            ?thing rdf:type ?type . 
                            }
        """%(keySearch)
    )
    result_set = set()
    for result in sparql.query().bindings:
        result = result['type'].value
        # find name of instance
        idx = result.rfind('#')
        if idx == -1:
            idx = result.rfind('/')
        result_type = result[idx+1:]
        result_set.add(result_type)
    return result_set

In [ ]:
def insert_instance(name, types, label):
    count = 0
    for result in types:
        # if type in onto.classes
        onto_type = dict_classes.get(result)
        if onto_type is not None:
            new_instance = onto_type(name, namespace=onto)
            print(type(new_instance), new_instance.name, label)
    return count

In [ ]:
# Lấy tất cả các class trong onto
dict_classes = dict()
for clazz in list(onto.classes()):
    dict_classes[clazz.name] = clazz
dict_classes

In [ ]:
df_ne_sub

In [ ]:
df_ne_sub['name'] = df_ne_sub['name'].apply(lambda x: x.replace('"', ''))

In [ ]:
query_type(' United States')

In [ ]:
df_sample = df_ne_sub.sample(n=10000, random_state=42)

In [ ]:
df_sample

In [ ]:
from tqdm import tqdm
for index, ne in tqdm(df_sample.iterrows()):
    types = query_type(ne['name'])
    val = ''
    for result in types:
        onto_type = dict_classes.get(result)
        if onto_type is not None:
            val += result + ', '
            onto_type(ne['name'], namespace=onto)
    ne['result'] = val

In [ ]:
df_sample.to_csv('/content/drive/MyDrive/Colab Notebooks/Đồ án/ne_travel.csv')

onto.save('/content/drive/MyDrive/Colab Notebooks/Đồ án/spacy_travel.owl')

In [ ]:
print(df_sample[df_sample['result'] != '']["result"].value_counts())
print(len(df_sample[df_sample['result'] != '']), 'inserted')

In [ ]:
df_sample.loc[:, ['name', 'label']].sample(10)